In [60]:
# If you ae new to go last cell

In [1]:
from catboost import CatBoostClassifier, Pool
import pandas as pd
# from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import StratifiedKFold,GridSearchCV
from sklearn.metrics import accuracy_score, classification_report,recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt

In [20]:
# Training Data (CC undersampling)===Testing Data (CC undersampling)
train_df = pd.read_csv(".././Training-Testing Dataset/Training Data (RWS Oversampling).csv")
test_df = pd.read_csv(".././Training-Testing Dataset/Testing Data (Representative).csv")
# test_df = pd.read_csv(".././Training-Testing Dataset/Testing Data (CC undersampling).csv")

X_train = train_df.drop(columns=["target"])
Y_train = train_df["target"]
X_test = test_df.drop(columns=["target"])
Y_test = test_df["target"]


print(Y_train.value_counts())
print(Y_test.value_counts())


target
Medium_Yes    4492
Low_No        4492
Medium_No     4492
High_No       4492
Low_Yes       4492
High_Yes      4492
Name: count, dtype: int64
target
Medium_No     1123
High_No        672
Low_No         455
Medium_Yes     127
High_Yes        76
Low_Yes         47
Name: count, dtype: int64


In [21]:
categorical_features = [col for col in X_train.columns if X_train[col].dtype == 'object']

classes = np.unique(Y_train)
class_weights = compute_class_weight(class_weight="balanced", classes=classes,y=Y_train)
class_weights_dict = dict(zip(classes, class_weights))
print(class_weights_dict)

for col in categorical_features:
    X_train[col] = X_train[col].astype('object')

    X_test[col] = X_test[col].astype('object')

# Y_train["target"] = Y_train["target"].astype(str)
# Y_test["target"] = Y_test["target"].astype(str)

train_pool = Pool(X_train,Y_train,cat_features=categorical_features)
test_pool = Pool(X_test,Y_test,cat_features=categorical_features)

{'High_No': np.float64(1.0), 'High_Yes': np.float64(1.0), 'Low_No': np.float64(1.0), 'Low_Yes': np.float64(1.0), 'Medium_No': np.float64(1.0), 'Medium_Yes': np.float64(1.0)}


In [22]:
model = CatBoostClassifier(
    iterations=3000,
    depth=12,
    learning_rate=0.03,
    l2_leaf_reg=0.8,
    loss_function="MultiClass",
    eval_metric="MultiClass", 
    early_stopping_rounds=200,
    verbose=50,
    random_state=42,
)

In [23]:
print("\nStarting CatBoost Training (Optimizing Weighted MultiClass Loss)...")
model.fit(train_pool,eval_set=test_pool)

y_pred_labels = model.predict(X_test).flatten()

# Evaluation: Still use scikit-learn for the final, explicit Macro Recall score
macro_recall = recall_score(Y_test, y_pred_labels, average='macro')
weighted_f1 = f1_score(Y_test, y_pred_labels, average='weighted')

print(f"Best Iteration: {model.get_best_iteration()}")
print(f"Macro Recall Score (The Goal Metric): {macro_recall * 100:.2f}%")
print(f"Weighted F1 Score: {weighted_f1 * 100:.2f}%")


Starting CatBoost Training (Optimizing Weighted MultiClass Loss)...
0:	learn: 1.7624646	test: 1.7778432	best: 1.7778432 (0)	total: 1.79s	remaining: 1h 29m 18s
50:	learn: 1.2776945	test: 1.5193119	best: 1.5193119 (50)	total: 1m 24s	remaining: 1h 21m 30s
100:	learn: 1.1515570	test: 1.4458944	best: 1.4458944 (100)	total: 2m 29s	remaining: 1h 11m 39s
150:	learn: 1.0820153	test: 1.4086873	best: 1.4086873 (150)	total: 3m 29s	remaining: 1h 5m 47s
200:	learn: 1.0089795	test: 1.3671017	best: 1.3671017 (200)	total: 4m 25s	remaining: 1h 1m 30s
250:	learn: 0.9553425	test: 1.3375847	best: 1.3375847 (250)	total: 5m 25s	remaining: 59m 19s
300:	learn: 0.8933737	test: 1.3029504	best: 1.3029504 (300)	total: 6m 15s	remaining: 56m 5s
350:	learn: 0.8031502	test: 1.2485988	best: 1.2485988 (350)	total: 7m 18s	remaining: 55m 5s
400:	learn: 0.6810854	test: 1.1542443	best: 1.1542443 (400)	total: 8m 34s	remaining: 55m 36s
450:	learn: 0.5751065	test: 1.0610725	best: 1.0610725 (450)	total: 9m 49s	remaining: 55m 3

In [12]:
model.save_model("catboost_marriage_model.cbm")

In [5]:
test_df_cc = pd.read_csv(".././Training-Testing Dataset/Testing data Undersampled(All).csv")

X_test_cc = train_df.drop(columns=["target"])
Y_test_cc = train_df["target"]


for col in categorical_features:
    X_test_cc[col] = X_test_cc[col].astype(str)

test_pool = Pool(X_test_cc,Y_test_cc,cat_features=categorical_features)

In [6]:
y_pred = model.predict(X_test_cc)

print("Accuracy rate ", accuracy_score(Y_test_cc,y_pred)*100)

Accuracy rate  44.925


In [7]:
test_df_cc = pd.read_csv(".././Training-Testing Dataset/Testing Data CC undersampling.csv")

X_test_cc = train_df.drop(columns=["target"])
Y_test_cc = train_df["target"]


for col in categorical_features:
    X_test_cc[col] = X_test_cc[col].astype(str)

test_pool = Pool(X_test_cc,Y_test_cc,cat_features=categorical_features)

y_pred = model.predict(X_test_cc)

print("Accuracy rate ", accuracy_score(Y_test_cc,y_pred)*100)

Accuracy rate  44.925


In [8]:
model = CatBoostClassifier()
model = model.load_model("catboost_marriage_model.cbm")
test_df_cc = pd.read_csv(".././Training-Testing Dataset/Testing data Undersampled(All).csv")

X_test_cc = test_df_cc.drop(columns=["target"])
Y_test_cc = test_df_cc["target"]


for col in categorical_features:
    X_test_cc[col] = X_test_cc[col].astype(str)

test_pool = Pool(X_test_cc,Y_test_cc,cat_features=categorical_features)

y_pred = model.predict(X_test_cc)

print("Accuracy rate ", accuracy_score(Y_test_cc,y_pred)*100)

Accuracy rate  80.31914893617021


In [14]:
features = X_test_cc.columns

for i,j in zip(features, model.get_feature_importance()):
    print(f"{i} : {j:0.2f}")

Age_at_Marriage : 14.14
Gender : 10.71
Education_Level : 13.79
Caste_Match : 6.90
Religion : 11.40
Urban_Rural : 9.93
Income_Level : 11.63
Spouse_Working : 10.79
Inter-Caste : 6.69
Inter-Religion : 4.02
